<a href="https://colab.research.google.com/github/Matthew-Dickson/COS801Assignment/blob/main/COS801Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>Some usefull information:</h2>


1.   https://www.kaggle.com/code/hsankesara/news-classification-using-han/notebook
2.   https://medium.com/analytics-vidhya/hierarchical-attention-networks-d220318cf87e



In [1]:
import pandas as pd
import requests, zipfile, io

<h2>Dataset section </h2>
Datasets can be found here: https://archive.ics.uci.edu/ml/datasets.php

In [2]:
zip_file_url =  'https://archive.ics.uci.edu/ml/machine-learning-databases/00454/dataset.zip' 

# Download data set and place in current directory under content directory
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("./content")
#testing

In [3]:
# Read training csv and place in panda data frame 
df = pd.read_csv(
    './content/dataset/Gungor_2018_VictorianAuthorAttribution_data-train.csv',
     encoding='latin-1'  
)

In [4]:
df.head()

,text,author
0,ou have time to listen i will give you the ent...,1
1,wish for solitude he was twenty years of age a...,1
2,and the skirt blew in perfect freedom about th...,1
3,of san and the rows of shops opposite impresse...,1
4,an hour s walk was as tiresome as three in a s...,1
